# **Final Project**
# Cleaning and Extract Dataset


### **1. Dataset Introduction**
**Dataset Name:** UK Online Retail Sales and Customer Transaction Data

This data set provides an in-depth look at transactions, product details, and customer information documented by an online retail company based in the UK. The scope of the data spans vastly, from granular details about each product sold to extensive customer data sets from different countries.

**Description:**

- **InvoiceNo**: A 6-digit number uniquely assigned to each transaction. If the number is prefixed with 'c', it indicates a cancellation.
- **StockCode**: A unique identifier for each product sold by the retailer.integral numbers
- **Description**: The name or a brief description of the product.
- **Quantity**: The number of units of the product sold in each transaction.
- **InvoiceDate**: The date and time when the transaction was made.
- **UnitPrice**: The price per unit of the product in sterling.
- **Country**: The country where the customer resides.

Credit: UCI - data.world

### **Objective**:
- Cleaning dataset
- Explore consumer behavior among wholesalers, facilitating analyses in inventory management and transactional trends within the online retail domain.

## **2. Data Cleaning and Extracting**

### **2.1 Import Data**

In [ ]:
# import data by pandas

import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/3. Data Analysis - Course/Phuc - Final Project/online_retail - UK real data.csv')

df.head()


,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### **2.2 Check data information, missing values and duplicates**

In [ ]:
# Check data information

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   index        541909 non-null  int64  
 1   InvoiceNo    541909 non-null  object 
 2   StockCode    541909 non-null  object 
 3   Description  540455 non-null  object 
 4   Quantity     541909 non-null  int64  
 5   InvoiceDate  541909 non-null  object 
 6   UnitPrice    541909 non-null  float64
 7   CustomerID   406829 non-null  float64
 8   Country      541909 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 37.2+ MB


In [ ]:
# Check missing value

df.isnull().sum()

,0
index,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [ ]:
# Drop dupliates

df.drop_duplicates(inplace=True)

#### **2.2.1 "Description" column missing values handling**

We will fill the "Description" based on the StockCode

In [ ]:
# "Description" missing values handling

# Create a map from StockCode to Description
desc_map = df.dropna(subset=['Description']).drop_duplicates('StockCode').set_index('StockCode')['Description']

# Using above map to fill the missing desciption
df['Description'] = df.apply(
    lambda row: desc_map[row['StockCode']] if pd.isnull(row['Description']) and row['StockCode'] in desc_map else row['Description'],
    axis=1
)


In [ ]:
# Check "Description" column again

df.isnull().sum()

,0
index,0
InvoiceNo,0
StockCode,0
Description,112
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


After filling in the missing descriptions, there are still 112 missing values remaining.

Given that this represents a very small portion of the dataset (112 out of 541,909), it's unlikely to significantly affect our analysis.

Therefore, we will remove these rows from the data.

In [ ]:
# Remove the missing Description values

df = df.dropna(subset=['Description'])

#### **2.2.2 Customer ID missing values handling**

To ensure consistency and serve the purpose of customer behavior analysis, data is filtered to only keep transactions with CustomerID information.

In [ ]:
# Remove missing Customer ID

df = df.dropna(subset=['CustomerID'])

### **2.3 Data Type Format**

In [ ]:
# Check Data Type

df.dtypes

,0
index,int64
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,object
UnitPrice,float64
CustomerID,float64
Country,object


In [ ]:
# Convert Invoice Date column to datetime

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Convert Customer ID column to integer

df['CustomerID'] = df['CustomerID'].astype(int)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   index        406829 non-null  int64         
 1   InvoiceNo    406829 non-null  object        
 2   StockCode    406829 non-null  object        
 3   Description  406829 non-null  object        
 4   Quantity     406829 non-null  int64         
 5   InvoiceDate  406829 non-null  datetime64[ns]
 6   UnitPrice    406829 non-null  float64       
 7   CustomerID   406829 non-null  int64         
 8   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 31.0+ MB


### **2.4 Cancellation Transaction Handling**

In [ ]:
# Remove cancellation transactions

df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

### **2.5 Create "Total Price" Column**

In [ ]:
# Create Total Price Column

df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

<ipython-input-13-693e33bdd8f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


### **2.6 Filter the suitable data**

Remove the rows where Quantity <= 0 or UnitPrice <=0

In [ ]:
# Remove the rows where Quantity <= 0 or UnitPrice <=0

df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

Remove the suspicious descriptions.

After checking the data again. I see there is some description which are not seem like a product but fees: "POSTAGE", "Manual", "DOTCOM POSTAGE", "CARRIAGE", "Next Day Carriage", "Bank Charges". We will remove these rows

In [ ]:
# Define the list of suspicious descriptions

non_product_descriptions = [
    "Manual",
    "POSTAGE",
    "DOTCOM POSTAGE",
    "CARRIAGE",
    "Next Day Carriage",
    "Bank Charges"
]

# Filter the DataFrame to exclude those rows

df = df[~df['Description'].isin(non_product_descriptions)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396261 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   index        396261 non-null  int64         
 1   InvoiceNo    396261 non-null  object        
 2   StockCode    396261 non-null  object        
 3   Description  396261 non-null  object        
 4   Quantity     396261 non-null  int64         
 5   InvoiceDate  396261 non-null  datetime64[ns]
 6   UnitPrice    396261 non-null  float64       
 7   CustomerID   396261 non-null  int64         
 8   Country      396261 non-null  object        
 9   TotalPrice   396261 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 33.3+ MB


## Aggregate data by CustomerID.

In [ ]:
customer_df = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',                 # total transactions
    'Quantity': 'sum',                      # total quantity purchased
    'UnitPrice': 'mean',                    # average price per item
    'InvoiceDate': ['min', 'max'],          # first & last purchase dates
    'Country': lambda x: x.mode()[0]        # most frequent country
})

customer_df.columns = ['TotalTransactions', 'TotalQuantity', 'AvgUnitPrice',
                       'FirstPurchaseDate', 'LastPurchaseDate', 'Country']

# Calculate Total Spending separately
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
total_spent = df.groupby('CustomerID')['TotalPrice'].sum()

customer_df['TotalSpent'] = total_spent


In [ ]:
customer_df.to_csv('customerID_based_dataset.csv', index=True)

from google.colab import files
files.download('customerID_based_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Now, the data is clean and ready to use**

## Calculate the Median AOV

In [ ]:
# Step 1: Create total price column if not yet
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Step 2: Calculate total value per invoice (order)
invoice_values = df.groupby('InvoiceNo')['TotalPrice'].sum().reset_index()
invoice_values.columns = ['InvoiceNo', 'OrderValue']


In [ ]:
median_aov = invoice_values['OrderValue'].median()
print(f"Median AOV: £{median_aov:.2f}")


Median AOV: £302.20
